In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes
!pip install -qqq torch
!pip install -qqq tensorflow
!pip install -qqq keras
!pip install -qqq transformers
!pip install -qqq peft
!pip install -qqq datasets
!pip install -qqq loralib
!pip install -qqq einops
!pip install sentence_transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
import datasets
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import notebook_login, list_datasets
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
    AutoPeftModelForCausalLM
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoModel
)

from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
notebook_login()

# Chemistry Only Model

In [4]:
# Load model directly
MODEL_NAME = "anishhs001/chemistry-falcon-7b"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

config = PeftConfig.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, MODEL_NAME)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.45M [00:00<?, ?B/s]

In [5]:

data = load_dataset("camel-ai/chemistry")

datset_req = data['train'].to_pandas()

train_set, test_set = train_test_split(datset_req, test_size=0.01, random_state=42)

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
%%time
device = "cuda:0"

# Load Sentence Transformer model
similarity_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
accuracy_val = 0
def text_similarity_1(text1, text2):
    # Encode the input texts
    embeddings = similarity_model.encode([text1, text2])

    # Calculate cosine similarity between the embeddings
    similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()

    return similarity_score

# Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def text_similarity_2(text1, text2):
    # Encode the input texts
    embeddings = embed([text1, text2])

    # Calculate cosine similarity between the embeddings
    similarity_score = tf.reduce_sum(tf.multiply(embeddings[0], embeddings[1]), axis=-1).numpy()

    return similarity_score

def accuracy_calc(test_set, accuracy_val = 0):
  for i in range(len(test_set)):
    prompt = test_set['message_1'].iloc[i].strip()

    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids = encoding.input_ids,
          attention_mask = encoding.attention_mask,
          generation_config = generation_config
      )
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    predicted_val = tokenizer.decode(outputs[0], skip_special_tokens=True)
    accuracy_val += (text_similarity_1(test_set['message_2'].iloc[i], predicted_val)+text_similarity_2(test_set['message_2'].iloc[i], predicted_val))/2
  accuracy_avg = accuracy_val/len(test_set)
  return accuracy_avg
print('Model Accuracy using similarity scoring: ', accuracy_calc(test_set))

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model Accuracy using similarity scoring:  0.7995300731807947
CPU times: user 53min 28s, sys: 10.5 s, total: 53min 39s
Wall time: 54min 34s


# Generalized PCB Model

In [7]:
# Load model directly
MODEL_NAME = "anishhs001/generalist-falcon-7b"

config = PeftConfig.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, MODEL_NAME)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

adapter_config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/9.45M [00:00<?, ?B/s]

In [9]:
%%time
device = "cuda:0"

# Load Sentence Transformer model
similarity_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
accuracy_val = 0
def text_similarity_1(text1, text2):
    # Encode the input texts
    embeddings = similarity_model.encode([text1, text2])

    # Calculate cosine similarity between the embeddings
    similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()

    return similarity_score

# Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def text_similarity_2(text1, text2):
    # Encode the input texts
    embeddings = embed([text1, text2])

    # Calculate cosine similarity between the embeddings
    similarity_score = tf.reduce_sum(tf.multiply(embeddings[0], embeddings[1]), axis=-1).numpy()

    return similarity_score

def accuracy_calc(test_set, accuracy_val = 0):
  for i in range(len(test_set)):
    prompt = test_set['message_1'].iloc[i].strip()

    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
      outputs = model.generate(
          input_ids = encoding.input_ids,
          attention_mask = encoding.attention_mask,
          generation_config = generation_config
      )
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    predicted_val = tokenizer.decode(outputs[0], skip_special_tokens=True)
    accuracy_val += (text_similarity_1(test_set['message_2'].iloc[i], predicted_val)+text_similarity_2(test_set['message_2'].iloc[i], predicted_val))/2
  accuracy_avg = accuracy_val/len(test_set)
  return accuracy_avg
print('Model Accuracy using similarity scoring: ', accuracy_calc(test_set))

Model Accuracy using similarity scoring:  0.7951750646531582
CPU times: user 52min 33s, sys: 9.12 s, total: 52min 42s
Wall time: 53min 14s
